In [43]:
import dataloader
import numpy as np
import smd
import probit
import pandas as pd
import os

In [49]:
from importlib import reload

reload(probit)
reload(smd)
reload(dataloader)

<module 'dataloader' from 'C:\\Columbia\\Courses\\AdvancedML\\advml\\dataloader.py'>

In [3]:
sparseAdWeights,adIds = dataloader.loadAdWeightsAndIds()
adIdsRev,convertToAdInx = dataloader.getAdIdConverterFunction(adIds)

In [4]:
def findNearestNeighbor(ad, wAds):
    diffs = wAds - ad
    norms = np.linalg.norm(diffs, axis=1)
    newAdIndex = np.argmin(norms)
    #   print(newAdIndex)
    return newAdIndex

In [50]:
def simulate(wAds, model, userInx):       
    submod = smd.SubModDivGlobal(wAds,model)
    
    wAds = wAds.toarray()
    
    prevAds = np.empty(0)
    prevAdData = np.empty((0, wAds.shape[1]))
    print(prevAdData)

    total = np.zeros(wAds.shape[1])
    meanAd = np.zeros(wAds.shape[1])

    for i in range(0, 100):
        subset = submod.getSubSet(userInx)
        
        newAdIndex = findNearestNeighbor(meanAd, wAds[subset])
        print(subset)
        print(newAdIndex)
        submod.registerClick(subset[newAdIndex])

        prevAds = np.append(prevAds, subset[newAdIndex])
        prevAdData = np.append(prevAdData, np.array([wAds[subset[newAdIndex]]]))

        total = total + wAds[subset[newAdIndex]]
        meanAd = total/float(prevAdData.shape[0])

#         wAds[newAdIndex, :] = float("inf")
        #     print("NewAdIndex: {}, Total: {}, meanAd: {}".format(newAdIndex, total, meanAd))

    return prevAdData, prevAds

In [13]:
eventsDf = pd.read_csv('filtered_events.csv')

uniqUser = np.unique(eventsDf.uuid)
nUser = uniqUser.shape[0]

In [15]:
model = probit.createProbitModel(sparseAdWeights,nUser)

In [28]:
weights_filename = 'probit.h5'

if os.path.isfile(weights_filename):
    print("loading existing weights")
    model.load_weights(weights_filename)

loading existing weights


In [36]:
sparseAdWeights.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [52]:
prevAdData, prevAds = simulate(sparseAdWeights, model, 5)
prevAds

[]
[    68 126534 126535 126536 126537 126538]
1
[126534 126535 126536 126537 126538 126539]
0
[126534 126535 126536 126537 126538 126539]
0
[126534 126535 126536 126537 126538 126539]
0
[126534 126535 126536 126537 126538 126539]
0
[126534 126535 126536 126537 126538 126539]
0
[126534 126535 126536 126537 126538 126539]
0
[126534 126535 126536 126537 126538 126539]
0


KeyboardInterrupt: 

# Metrics

In [ ]:
def accuracyAtK(adClickInx, wAds, model, userInx, k):
    probs = self.model.predict([
            np.array([userInx]*wAds.shape[0]),
            np.arange(wAds.shape[0])
        ],batch_size=50000).ravel()
    
    topAds = sorted(range(0, len(probs)), key=lambda x: prob[x], reverse=True)[:k]
    
    common = set(adClickInx).intersection(set(topAds))
    return len(common)/float(len(adClickInx))

In [ ]:
def meanClickProbability(subSets, model):
    probs = self.model.predict([
            np.array([userInx]*wAds.shape[0]),
            np.arange(wAds.shape[0])
        ],batch_size=50000).ravel()
    
    subSetProbs = np.sum(probs[subset])